In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train_identity = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_identity.csv")
train_transaction = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_transaction.csv")
test_identity = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_identity.csv")
test_transaction = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_transaction.csv")

In [ ]:
train_identity.head(5)

In [ ]:
train_transaction.head(5)

In [ ]:
train_identity.shape

In [ ]:
train_transaction.shape

In [ ]:
train_transaction.columns

In [ ]:
train_identity.columns

In [ ]:
test_transaction.shape

In [ ]:
test_identity.shape

**Check how many fraud cases are there**

In [ ]:
fc = train_transaction['isFraud'].value_counts(normalize=True).to_frame()
fc.plot.bar()

In [ ]:
fc

* there is a class imbalace problem.

**Merge the transaction and identity data**

In [ ]:
train=train_transaction.merge(train_identity,how='left',left_index=True,right_index=True)
y_train=train['isFraud'].astype('uint8')
print('Train shape',train.shape)
del train_transaction,train_identity
print("Data set merged ")

In [ ]:
train.head(3)

**Merge the test data **

In [ ]:
test=test_transaction.merge(test_identity,how='left',left_index=True,right_index=True)
print('Test shape',test.shape)
del test_transaction,test_identity
print("Test Data set merged ")

In [ ]:
test.head(3)


In [ ]:
%%time
# From kernel https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
# WARNING! THIS CAN DAMAGE THE DATA 
def reduce_mem_usage2(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


In [ ]:
%%time
train = reduce_mem_usage2(train)

In [ ]:
%%time
test = reduce_mem_usage2(test)

**Data Imbalance**

One of the techniques to overcome imbalance is to use "Oversampling minority clas"

Oversampling is defined as adding more copies of the minority class. Oversampling can be a good choice when you don’t have a ton of data to work with.




In [ ]:
not_fraud=train[train.isFraud==0]
fraud=train[train.isFraud==1]

In [ ]:
from sklearn.utils import resample

# upsample minority
fraud_upsampled = resample(fraud,
                          replace=True, # sample with replacement
                          n_samples=len(not_fraud), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
X = pd.concat([not_fraud, fraud_upsampled])

# check new class counts
X.isFraud.value_counts()

In [ ]:
len(X)

In [ ]:
y = X['isFraud']

In [ ]:
#X = X.drop(['isFraud'], axis=1)

**Convert TransactionDT to day time**

In [ ]:
X["TransactionDay"] = X["TransactionDT"] // (24*60*60)
X["TransactionWeek"] = X["TransactionDay"] // 7

![](http://)![](http://)**Analyze fraud on ProductCD**

In [ ]:
prod = list(set(X["ProductCD"]))
print(prod)

In [ ]:
X.groupby("ProductCD")['isFraud'].value_counts().to_frame().plot.bar()


more number of frauds are for product **"C" and "W"**

In [ ]:
X.groupby(['card4', 'card3'])['isFraud'].value_counts().to_frame().plot.bar()


In [ ]:
X.groupby(['card4'])['isFraud'].value_counts().to_frame().plot.bar()


**Most number of frauds are for**
* Visa
* Mastercard

In [ ]:
X.groupby(['card4', 'ProductCD'])['isFraud'].value_counts().to_frame().plot.bar()


 **Most number of fraud cases are seen for **
   * Visa, Master card for 
   * Product "C" and "W" 

**Analyze for Fraud Transaction amount**

In [ ]:
fraud = X[X['isFraud'] == 1]
print("max trans amount happend during fraud:",max(fraud["TransactionAmt"]))

In [ ]:
print("Min trans amount happend during fraud:",min(fraud["TransactionAmt"]))

In [ ]:
fraud[fraud["TransactionAmt"] == 0.292]

**Analysis on Address**

In [ ]:
X['addr1']

In [ ]:
X['addr2']

In [ ]:
fraud.groupby(['addr1', 'addr2'])['isFraud'].value_counts().to_frame().plot.bar()


 **Columns with null values**
 
 Drop the columns which has more than 50% of null values

In [ ]:
null_percent = train.isnull().sum()/train.shape[0]*100

cols_to_drop = np.array(null_percent[null_percent > 50].index)

cols_to_drop

In [ ]:
X = X.drop(cols_to_drop, axis=1)
test = test.drop(cols_to_drop,axis=1)

In [ ]:
X.columns


In [ ]:
null_percent = test.isnull().sum()/X.shape[0]*100
null_percent[null_percent > 0]


In [ ]:
cols_to_drop_again = np.array(null_percent[null_percent > 0.001].index)

cols_to_drop_again

In [ ]:
X = X.drop(cols_to_drop_again, axis=1)
test = test.drop(cols_to_drop_again,axis=1)

In [ ]:
list(X.columns)

Replace the nan with mode values

**Get more visualizations for email domanin, transaction amt,  **

** looking at the distribution of Transaction amount"

In [ ]:
 sns.distplot(a=X["TransactionAmt"])

It is exponential distribution , doing log normal on it

In [ ]:
log_trans = X['TransactionAmt'].apply(np.log)
sns.distplot(a=log_trans)

Applying log , Close to normal distribution, apply it to the column

In [ ]:
X['TransactionAmt'] = X['TransactionAmt'].apply(np.log)
test['TransactionAmt'] = test['TransactionAmt'].apply(np.log)

In [ ]:
cols_to_drop = ['TransactionDT','TransactionID_x']
X = X.drop(cols_to_drop, axis=1)

**Find the categorical and numerical data**

In [ ]:
cat_data = X.select_dtypes(include='object')
num_data = X.select_dtypes(exclude='object')

cat_cols = cat_data.columns.values
num_cols = num_data.columns.values

List of Categorical data

In [ ]:
cat_cols

**Label encode the categorical variables**

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm



In [ ]:
for i in tqdm(cat_cols): 
    label = LabelEncoder()
    label.fit(list(X[i].values)+list(test[i].values))
    X[i] = label.transform(list(X[i].values))
    test[i] = label.transform(list(test[i].values))

In [ ]:
X.shape

In [ ]:
X.head(10)

**Find the correlaiton between variables**

In [ ]:
corr = X.corr()

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(corr)

In [ ]:
col_corr = set()
for i in range(len(corr.columns)):
    for j in range(i):
        if (corr.iloc[i, j] >= 0.95) and (corr.columns[j] not in col_corr):
            colname = corr.columns[i] # getting the name of column
            col_corr.add(colname)

In [ ]:
col_corr

Get rid of highly correlated columns 

In [ ]:
final_columns = []

cols = X.columns

for i in cols:
    if i in col_corr:
        continue
    else:
        final_columns.append(i)
        
        

In [ ]:
final_columns

> **Take the coefficients with less correlation in to new data frame**

In [ ]:
X_final = X[final_columns]

In [ ]:
test_final = test[final_columns]

In [ ]:
print(X_final.shape)
print(test_final.shape)

**Model**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

In [ ]:
scaler.fit(X_final)

In [ ]:
dfX = scaler.transform(X_final)


In [ ]:
feature_name = X_final.columns

In [ ]:
dfX = pd.DataFrame(dfX, columns=feature_name)


In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, BatchNormalization, Dropout, Flatten, Input
from keras import backend as K
import keras

n_features = dfX.shape[1]
dim = 15

def build_model(dropout_rate=0.15, activation='tanh'):
    main_input = Input(shape=(n_features, ), name='main_input')
    
    x = Dense(dim*2, activation=activation)(main_input)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    
    x = Dense(dim*2, activation=activation)(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate/2)(x)
    
    x = Dense(dim, activation=activation)(x)
    x = Dropout(dropout_rate/4)(x)

    encoded = Dense(2, activation='tanh')(x)

    input_encoded = Input(shape=(2, ))
    
    x = Dense(dim, activation=activation)(input_encoded)
    x = Dense(dim, activation=activation)(x)
    x = Dense(dim*2, activation=activation)(x)
    
    decoded = x = Dense(n_features, activation='linear')(x)

    encoder = Model(main_input, encoded, name="encoder")
    decoder = Model(input_encoded, decoded, name="decoder")
    autoencoder = Model(main_input, decoder(encoder(main_input)), name="autoencoder")
    return encoder, decoder, autoencoder

K.clear_session()
c_encoder, c_decoder, c_autoencoder = build_model()
c_autoencoder.compile(optimizer='nadam', loss='mse')

c_autoencoder.summary()

In [ ]:
dfX.shape

In [ ]:
%%time
epochs = 50
batch_size = 9548
history = c_autoencoder.fit(dfX, y,
                    epochs=epochs,
                    batch_size=batch_size,
                    shuffle=True,
                    verbose=1)

loss_history = history.history['loss']
plt.figure(figsize=(10, 5))
plt.plot(loss_history);

In [ ]:
params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47,
         }

In [ ]:
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
seed = 123
kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=seed)
pred_test_full =0
cv_score =[]
i=1
predictions = np.zeros(test_final.shape[0])
print('5 Fold Stratified Cross Validation')
print('-----------------------------------')
for train_index,test_index in kf.split(X_final,y):
    print('{} of KFold {}'.format(i,kf.n_splits))
    xtr,xv = X_final.loc[train_index],X_final.loc[test_index]
    ytr,yv = y.loc[train_index],y.loc[test_index]
    
    #model
    #clf = LogisticRegression(C=2)
    #clf.fit(xtr,ytr)
    #clf = CatBoostClassifier(task_type='GPU', eval_metric='AUC', loss_function='Logloss', use_best_model=True,
    #                      silent=True, class_weights=[0.05, 0.95],
    #                     random_state=42, iterations=5000, od_type='Iter', od_wait=200, grow_policy='Lossguide', max_depth=8)
    dtrain = lgb.Dataset(xtr, label=ytr)
    dvalid = lgb.Dataset(xv, label=yv)

    clf = lgb.train(params, dtrain, 10000, valid_sets = [dtrain, dvalid], verbose_eval=200, early_stopping_rounds=500)
    
    y_pred_valid = clf.predict(xv)
    score = roc_auc_score(yv,clf.predict(xv))
    print('ROC AUC score:',score)
    cv_score.append(score)    
    pred_test = clf.predict(test_final)/5
    pred_test_full +=pred_test
    i+=1
    print('-------------------------------------')

In [ ]:
print('Mean AUC Score for CatBoost : {}'.format(np.array(score).mean()))

In [ ]:
sub = pd.read_csv('/kaggle/input/ieee-fraud-detection/sample_submission.csv')

In [ ]:
sub['isFraud'] = pred_test_full

In [ ]:
sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)